In [379]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)


In [380]:
train = pd.read_csv('train_cleaned.csv')
test = pd.read_csv('test_cleaned.csv')
target = pd.read_csv('training_set_labels.csv')

train.drop('Unnamed: 0',inplace=True,axis=1)
test.drop('Unnamed: 0',inplace=True,axis=1)

#total = pd.concat([train,test])
total = train.append(test, ignore_index = True)

In [381]:
target.head(1)

,id,status_group
0,69572,functional


In [382]:
train = train.drop(['id'], axis = 1) #Dropping the id from the train dataset as it will not help in any way in predicting the target
test_ids = test['id'] #Keeping the test ids in a separate file for final prediction
test = test.drop(['id'], axis = 1) #Again dropping the ids from the test dataset as they are not going to help in final prediction
target = target.drop(['id'], axis = 1) #Similarly dropping the ids from the train labels dataset
total = total.drop(['id'], axis = 1)

In [383]:
target['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [384]:
proportion = (target['status_group'].value_counts()/target['status_group'].value_counts().sum())
proportion

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [385]:
total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74250 entries, 0 to 74249
Data columns (total 29 columns):
amount_tsh           74250 non-null float64
funder               69746 non-null object
gps_height           74250 non-null float64
installer            69718 non-null object
longitude            74250 non-null float64
latitude             74250 non-null float64
basin                74250 non-null object
subvillage           73780 non-null object
region               74250 non-null object
region_code          74250 non-null int64
district_code        74250 non-null int64
lga                  74250 non-null object
ward                 74250 non-null object
population           74250 non-null float64
public_meeting       70095 non-null object
scheme_management    69404 non-null object
scheme_name          38992 non-null object
permit               70457 non-null object
construction_year    74250 non-null float64
extraction_type      74250 non-null object
management           74250 

In [386]:
total.columns

Index(['amount_tsh', 'funder', 'gps_height', 'installer', 'longitude',
       'latitude', 'basin', 'subvillage', 'region', 'region_code',
       'district_code', 'lga', 'ward', 'population', 'public_meeting',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'management', 'management_group', 'payment',
       'payment_type', 'water_quality', 'quantity', 'source',
       'waterpoint_type', 'operation_years'],
      dtype='object')

In [387]:
total = total.drop(['funder','payment_type','scheme_name','subvillage','region','region_code','district_code','lga','ward',
              'public_meeting','water_quality'], axis = 1)

In [388]:
total['population'] = total['population'].fillna(total['population'].median())
if total['construction_year'].value_counts().index[0]==0:
    total['construction_year'] = total['construction_year'].fillna(total['construction_year'].value_counts().index[1])
else:
    total['construction_year'] = total['construction_year'].fillna(total['construction_year'].value_counts().index[0])

In [389]:
cols = ['installer','scheme_management','permit']
for col in cols:
    if total[col].value_counts().index[0]==0:
        total[col] = total[col].fillna(total[col].value_counts().index[1])
    else:
        total[col] = total[col].fillna(total[col].value_counts().index[0])

In [390]:
total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74250 entries, 0 to 74249
Data columns (total 18 columns):
amount_tsh           74250 non-null float64
gps_height           74250 non-null float64
installer            74250 non-null object
longitude            74250 non-null float64
latitude             74250 non-null float64
basin                74250 non-null object
population           74250 non-null float64
scheme_management    74250 non-null object
permit               74250 non-null bool
construction_year    74250 non-null float64
extraction_type      74250 non-null object
management           74250 non-null object
management_group     74250 non-null object
payment              74250 non-null object
quantity             74250 non-null object
source               74250 non-null object
waterpoint_type      74250 non-null object
operation_years      74250 non-null int64
dtypes: bool(1), float64(6), int64(1), object(10)
memory usage: 9.7+ MB


In [391]:
#divide data again into train and test

train = total.iloc[:59400,:]
test = total.iloc[59400:,:]

In [392]:
#installer dataframe & dummified target
train_installer_target = pd.concat([train['installer'],target], axis=1)
target_dummies = pd.get_dummies(train_installer_target['status_group'])
train_installer_target = train_installer_target.drop(['status_group'], axis = 1)
train_installer_target = pd.concat([train_installer_target,target_dummies], axis=1)
train_installer_target.head(2)

,installer,functional,functional needs repair,non functional
0,Roman,1,0,0
1,GRUMETI,1,0,0


In [393]:
train_installer_target_prop = pd.concat([train_installer_target.groupby('installer').mean(),train_installer_target.groupby('installer').count()], axis = 1).iloc[:,:-2]
train_installer_target_prop.columns = ['functional','functional needs repair','non functional','count']
train_installer_target_prop.sort_values(by='count',ascending=False).head()

,functional,functional needs repair,non functional,count
installer,,,,
DWE,0.542955,0.097925,0.359120,21057
Government,0.293151,0.140274,0.566575,1825
RWE,0.252073,0.113599,0.634328,1206
Commu,0.683019,0.030189,0.286792,1060
DANIDA,0.516190,0.079048,0.404762,1050


In [394]:
train_installer_target_prop['category'] = np.zeros((train_installer_target_prop.shape[0],1))
train_installer_target_prop['category'].loc[train_installer_target_prop['functional'] > 0.54] = 'functional'
train_installer_target_prop['category'].loc[train_installer_target_prop['non functional'] > 0.38] = 'non functional'
train_installer_target_prop['category'].loc[train_installer_target_prop['functional needs repair'] > 0.05] = 'functional needs repair'

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [395]:
train['installer_category'] = train.installer.map(train_installer_target_prop.category)
test['installer_category'] = test.installer.map(train_installer_target_prop.category)
test['installer_category'] = test['installer_category'].fillna(test['installer_category'].value_counts().index[0])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [396]:
total = train.append(test, ignore_index = True)

In [397]:
total = total.drop(['installer'], axis=1)

In [398]:
total.describe(include = ['O'])

,basin,scheme_management,extraction_type,management,management_group,payment,quantity,source,waterpoint_type,installer_category
count,74250,74250,74250,74250,74250,74250,74250,74250,74250,74250
unique,9,12,18,12,5,7,5,10,7,3
top,Lake Victoria,VWC,gravity,vwc,user-group,never pay,enough,spring,communal standpipe,functional needs repair
freq,12871,50763,33263,50624,65538,31712,41522,21216,35628,47078


In [399]:
total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74250 entries, 0 to 74249
Data columns (total 18 columns):
amount_tsh            74250 non-null float64
gps_height            74250 non-null float64
longitude             74250 non-null float64
latitude              74250 non-null float64
basin                 74250 non-null object
population            74250 non-null float64
scheme_management     74250 non-null object
permit                74250 non-null bool
construction_year     74250 non-null float64
extraction_type       74250 non-null object
management            74250 non-null object
management_group      74250 non-null object
payment               74250 non-null object
quantity              74250 non-null object
source                74250 non-null object
waterpoint_type       74250 non-null object
operation_years       74250 non-null int64
installer_category    74250 non-null object
dtypes: bool(1), float64(6), int64(1), object(10)
memory usage: 9.7+ MB


In [400]:
total_dummies = pd.get_dummies(total)

In [401]:
total_dummies.describe()

,amount_tsh,gps_height,longitude,latitude,population,construction_year,operation_years,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,basin_Rufiji,basin_Ruvuma / Southern Coast,basin_Wami / Ruvu,scheme_management_Company,scheme_management_None,scheme_management_Other,scheme_management_Parastatal,scheme_management_Private operator,scheme_management_SWC,scheme_management_Trust,scheme_management_VWC,scheme_management_WUA,scheme_management_WUG,scheme_management_Water Board,scheme_management_Water authority,extraction_type_afridev,extraction_type_cemo,extraction_type_climax,extraction_type_gravity,extraction_type_india mark ii,extraction_type_india mark iii,extraction_type_ksb,extraction_type_mono,extraction_type_nira/tanira,extraction_type_other,extraction_type_other - mkulima/shinyanga,extraction_type_other - play pump,extraction_type_other - rope pump,extraction_type_other - swn 81,extraction_type_submersible,extraction_type_swn 80,extraction_type_walimi,extraction_type_windmill,management_company,management_other,management_other - school,management_parastatal,management_private operator,management_trust,management_unknown,management_vwc,management_water authority,management_water board,management_wua,management_wug,management_group_commercial,management_group_other,management_group_parastatal,management_group_unknown,management_group_user-group,payment_never pay,payment_other,payment_pay annually,payment_pay monthly,payment_pay per bucket,payment_pay when scheme fails,payment_unknown,quantity_dry,quantity_enough,quantity_insufficient,quantity_seasonal,quantity_unknown,source_dam,source_hand dtw,source_lake,source_machine dbh,source_other,source_rainwater harvesting,source_river,source_shallow well,source_spring,source_unknown,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,installer_category_functional,installer_category_functional needs repair,installer_category_non functional
count,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000
mean,1241.682018,1078.416653,35.091203,-5.795470,288.755872,1997.635111,14.140929,0.129859,0.085279,0.041199,0.108444,0.173347,0.150074,0.134505,0.075246,0.102047,0.018061,0.000013,0.013414,0.028606,0.017859,0.001657,0.001239,0.683677,0.047825,0.087488,0.046626,0.053535,0.029737,0.001455,0.000552,0.447987,0.040795,0.001818,0.024108,0.048862,0.137441,0.109118,0.000027,0.001360,0.007704,0.003825,0.080566,0.061791,0.000808,0.002047,0.011569,0.014586,0.001697,0.030020,0.033724,0.001414,0.009199,0.681805,0.015125,0.049670,0.041993,0.109199,0.061832,0.016283,0.030020,0.009199,0.882667,0.427098,0.017697,0.061549,0.140027,0.15173

In [402]:
total_dummies = total_dummies.drop(['construction_year'], axis=1)
total_dummies.shape

(74250, 95)

In [403]:
train = total_dummies.iloc[:59400,:]
test = total_dummies.iloc[59400:,:]

In [404]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [405]:
classifier = RandomForestClassifier(n_estimators=500, random_state = 8)


In [406]:
score = cross_val_score(classifier, train, np.ravel(target), scoring = 'accuracy',cv=5).mean()
print("Score: {}".format(score))

Score: 0.804158240399234


In [407]:

total_dummies.head()

,amount_tsh,gps_height,longitude,latitude,population,permit,operation_years,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,basin_Rufiji,basin_Ruvuma / Southern Coast,basin_Wami / Ruvu,scheme_management_Company,scheme_management_None,scheme_management_Other,scheme_management_Parastatal,scheme_management_Private operator,scheme_management_SWC,scheme_management_Trust,scheme_management_VWC,scheme_management_WUA,scheme_management_WUG,scheme_management_Water Board,scheme_management_Water authority,extraction_type_afridev,extraction_type_cemo,extraction_type_climax,extraction_type_gravity,extraction_type_india mark ii,extraction_type_india mark iii,extraction_type_ksb,extraction_type_mono,extraction_type_nira/tanira,extraction_type_other,extraction_type_other - mkulima/shinyanga,extraction_type_other - play pump,extraction_type_other - rope pump,extraction_type_other - swn 81,extraction_type_submersible,extraction_type_swn 80,extraction_type_walimi,extraction_type_windmill,management_company,management_other,management_other - school,management_parastatal,management_private operator,management_trust,management_unknown,management_vwc,management_water authority,management_water board,management_wua,management_wug,management_group_commercial,management_group_other,management_group_parastatal,management_group_unknown,management_group_user-group,payment_never pay,payment_other,payment_pay annually,payment_pay monthly,payment_pay per bucket,payment_pay when scheme fails,payment_unknown,quantity_dry,quantity_enough,quantity_insufficient,quantity_seasonal,quantity_unknown,source_dam,source_hand dtw,source_lake,source_machine dbh,source_other,source_rainwater harvesting,source_river,source_shallow well,source_spring,source_unknown,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,installer_category_functional,installer_category_functional needs repair,installer_category_non functional
0,6000.000000,1390.0,34.938093,-9.856322,109.0,False,12,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
1,537.500000,1399.0,34.698766,-2.147466,280.0,True,3,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
2,25.000000,686.0,37.460664,-3.821329,250.0,True,4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
3,321.428571,263.0,38.486161,-11.155298,58.0,True,27,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,1241.682018,1192.0,31.130847,-1.825359,200.0,True,11,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [408]:
model = RandomForestClassifier(n_estimators=500, random_state=8)
model.fit(train,np.ravel(target))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=False, random_state=8, verbose=0, warm_start=False)

In [409]:
test_predictions = model.predict(test)

In [410]:
submission = pd.DataFrame(test_predictions, columns=['status_group'])
submission.insert(0, 'id', test_ids)
submission.reset_index()
submission.to_csv('submission.csv', index = False)